In [22]:
import numpy as np
import torch
from smplx import MANO

comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
left_mano_path = '../models/mano_models/MANO_LEFT.pkl'
right_mano_path = '../models/mano_models/MANO_RIGHT.pkl'

left_model = MANO(model_path=left_mano_path,is_rhand=False,use_pca=False,flat_hand_mean=True).to(comp_device)
right_model = MANO(model_path=right_mano_path,is_rhand=True,use_pca=False,flat_hand_mean=True).to(comp_device)

In [23]:
data_number = 494
data_path = '../data/both57m/%d/data.npz'%(data_number)
data = np.load(data_path)
seq_len = data['seq_len']

hand_pose = torch.Tensor(data['hand_pose'].reshape((seq_len,-1))).to(comp_device)
left_hand_pose = hand_pose[:,:45]
right_hand_pose = hand_pose[:,45:]

left_joint_pos_list = []
right_joint_pos_list = []
left_extra_joints_idxs = left_model.vertex_joint_selector.extra_joints_idxs
right_extra_joints_idxs = right_model.vertex_joint_selector.extra_joints_idxs
finger_idxs = [0,1,2,3,5,6,7,9,10,11,13,14,15,17,18,19]
fingertip_idxs = [20,4,8,16,12]

for i in range(seq_len):
    left_output = left_model(hand_pose=left_hand_pose[i].unsqueeze(0),return_verts=True)
    left_finger_pos_output = left_output.joints.detach().cpu().numpy().squeeze()
    left_fingertip_pos_output = left_output.vertices[:,left_extra_joints_idxs,:].detach().cpu().numpy().squeeze()
    left_full_joint_pos = np.zeros((21,3))
    left_full_joint_pos[finger_idxs] = left_finger_pos_output
    left_full_joint_pos[fingertip_idxs] = left_fingertip_pos_output
    left_joint_pos_list.append(left_full_joint_pos)

    right_output = right_model(hand_pose=right_hand_pose[i].unsqueeze(0),return_verts=True)
    right_finger_pos_output = right_output.joints.detach().cpu().numpy().squeeze()
    right_fingertip_pos_output = right_output.vertices[:,right_extra_joints_idxs,:].detach().cpu().numpy().squeeze()
    right_full_joint_pos = np.zeros((21,3))
    right_full_joint_pos[finger_idxs] = right_finger_pos_output
    right_full_joint_pos[fingertip_idxs] = right_fingertip_pos_output
    right_joint_pos_list.append(right_full_joint_pos)

left_joint_pos_array = np.array(left_joint_pos_list)
right_joint_pos_array = np.array(right_joint_pos_list)

mano2mp = [0,17,18,19,20,1,2,3,4,5,6,7,8,13,14,15,16,9,10,11,12]
left_joint_pos_array = left_joint_pos_array[:,mano2mp]
right_joint_pos_array = right_joint_pos_array[:,mano2mp]

MANO_LEFT_UP = np.array([[0, -1, 0],[0, 0, -1],[1, 0, 0]])
MANO_RIGHT_UP = np.array([[0, -1, 0],[0, 0, 1],[-1, 0, 0]])
for i in range(seq_len):
    left_joint_pos_array[i] = np.matmul(MANO_LEFT_UP,left_joint_pos_array[i].T).T
    right_joint_pos_array[i] = np.matmul(MANO_RIGHT_UP,right_joint_pos_array[i].T).T

print("Left Joint Position shape: ", left_joint_pos_array.shape)
print("Right Joint Position shape: ", right_joint_pos_array.shape)

Left Joint Position shape:  (150, 21, 3)
Right Joint Position shape:  (150, 21, 3)


In [24]:
from pathlib import Path

from dex_retargeting.constants import RobotName, RetargetingType, HandType, get_default_config_path
from dex_retargeting.retargeting_config import RetargetingConfig
from dex_retargeting.seq_retarget import SeqRetargeting

robot_name = RobotName.ability
retargeting_type = RetargetingType.dexpilot

In [25]:
hand_type_left = HandType.left

config_path = get_default_config_path(robot_name, retargeting_type, hand_type_left)
robot_dir = Path().absolute().parent / "ability_hand" / "assets" 
RetargetingConfig.set_default_urdf_dir(str(robot_dir))
left_retargeting = RetargetingConfig.load_from_file(config_path).build()

left_ability_joints_list = []

for i in range(seq_len):
    retargeting_type = left_retargeting.optimizer.retargeting_type
    indices = left_retargeting.optimizer.target_link_human_indices
    origin_indices = indices[0, :]
    task_indices = indices[1, :]
    
    left_joint_pos = left_joint_pos_array[i]
    ref_value = left_joint_pos[task_indices, :] - left_joint_pos[origin_indices, :]
    left_qpos = left_retargeting.retarget(ref_value)
    left_ability_joints_list.append(left_qpos)

left_joint_names=left_retargeting.optimizer.robot.dof_joint_names
left_ability_joints_array = np.array(left_ability_joints_list)
print("Left Ability Joint Position shape: ", left_ability_joints_array.shape)

 Mimic joint adaptor enabled. The mimic joint tags in the URDF will be considered during retargeting.
To disable mimic joint adaptor, consider setting ignore_mimic_joint=True in the configuration. 
Left Ability Joint Position shape:  (150, 10)


In [26]:
hand_type_right = HandType.right

config_path = get_default_config_path(robot_name, retargeting_type, hand_type_right)
right_retargeting = RetargetingConfig.load_from_file(config_path).build()

right_ability_joints_list = []
for i in range(seq_len):
    retargeting_type = right_retargeting.optimizer.retargeting_type
    indices = right_retargeting.optimizer.target_link_human_indices
    origin_indices = indices[0, :]
    task_indices = indices[1, :]

    right_joint_pos = right_joint_pos_array[i]
    ref_value = right_joint_pos[task_indices, :] - right_joint_pos[origin_indices, :]
    right_qpos = right_retargeting.retarget(ref_value)
    right_ability_joints_list.append(right_qpos)

right_joint_names=right_retargeting.optimizer.robot.dof_joint_names
right_ability_joints_array = np.array(right_ability_joints_list)
print("Right Ability Joint Position shape: ", right_ability_joints_array.shape)

 Mimic joint adaptor enabled. The mimic joint tags in the URDF will be considered during retargeting.
To disable mimic joint adaptor, consider setting ignore_mimic_joint=True in the configuration. 
Right Ability Joint Position shape:  (150, 10)


In [27]:
import sys
sys.path.append('../utils')
from mujoco_parser import MuJoCoParserClass
from util import rpy2r,get_idxs,np_uv

xml_path = '../asset/kimlab_dualarms/scene_dualarms.xml'
env = MuJoCoParserClass(name='Dual Arms',rel_xml_path=xml_path,VERBOSE=False)
env.set_geom_color(
    rgba                = (0.2,0.2,0.2,0.9),
    body_names_to_color = [
        'robot1/end_link','robot1/index_L1','robot1/index_L2','robot1/index_tip',
        'robot1/middle_L1','robot1/middle_L2','robot1/middle_tip',
        'robot1/ring_L1','robot1/ring_L2','robot1/ring_tip',
        'robot1/pinky_L1','robot1/pinky_L2','robot1/pinky_tip',
        'robot1/thumb_L1','robot1/thumb_L2','robot1/thumb_tip',
        'robot2/end_link','robot2/index_L1','robot2/index_L2','robot2/index_tip',
        'robot2/middle_L1','robot2/middle_L2','robot2/middle_tip',
        'robot2/ring_L1','robot2/ring_L2','robot2/ring_tip',
        'robot2/pinky_L1','robot2/pinky_L2','robot2/pinky_tip',
        'robot2/thumb_L1','robot2/thumb_L2','robot2/thumb_tip'],
)

left_hand_joint_names = ['robot1/' + i for i in left_joint_names]
left_hand_idxs = get_idxs(env.joint_names,left_hand_joint_names)

right_hand_joint_names = ['robot2/' + i for i in right_joint_names]
right_hand_idxs = get_idxs(env.joint_names,right_hand_joint_names)

In [32]:
import time
env.init_viewer(viewer_title='Dual Arms',viewer_width=800,viewer_height=800,viewer_hide_menus=True)
env.update_viewer(azimuth=180,distance=1.7,elevation=-8.3,lookat=[0.02,-0.03,2.1])
env.reset()

q_init = np.zeros(env.model.nq)
q_init[5] = np.pi/2
q_init[21] = -np.pi/2

env.forward(q=q_init)
tick = 0
video = []
SAVE_VIDEO = True

while(env.is_viewer_alive() and tick < seq_len):
    left_ability_joints = left_ability_joints_array[tick]
    right_ability_joints = right_ability_joints_array[tick]
    ability_joints = np.concatenate((left_ability_joints,right_ability_joints))
    hand_idxs = np.concatenate((left_hand_idxs,right_hand_idxs))

    env.forward(q=ability_joints,joint_idxs=hand_idxs)
    env.render()
    env.plot_T(p=np.array([0,0,2]),R=np.eye(3),label='Tick: %d/%d'%(tick,seq_len),PLOT_AXIS=False)
    
    tick += 1
    if SAVE_VIDEO:
        video.append(env.grab_image())
    else:
        time.sleep(1/30)
        if tick == seq_len: tick = 0

env.close_viewer()

In [33]:
import mediapy
mediapy.show_video(video,fps=30)

This browser does not support the video tag.
